$\newcommand{\d}{\mathrm{d}}$
I'm too lazy to write all details, so I just write some interesting implementation details. Defintely it's not a tech report. 

# Questions that nobody cares

### Is it fast? How fast is it?

No, it's slow, compared to [primecount](https://github.com/kimwalisch/primecount). 

With that being said, it's not that slow as people might think. For example, with the help of LMFDB, it successfully calculates $\pi(10^{18}) = 24739954287740860$ in about 8.5min in Macbook Pro 13-inch (2017) with a single thread. As a comparison, `primecount` gives the same answer in about 100s in the same environment (`primecount 1e18 -t 1`), so the analytic method is ~5x slower.

### If a number is produced, is it guaranteed to be correct?

No. There are some possible reasons:

1. I added my own heuristics and traded correctness for speed. For example, I use heuristics to decide the interval to compute $\Delta$ (see technical details below).
2. Unlike [Platt], I don't use interval/ball arithmetic (MPFI/Arb) and don't do error analysis. I don't even use existing high-precision arithmetic library, so tracking the rounding error alone is nearly impossible.
3. Also for the logic code: it's not tested, so there might be bugs.

### Why Rust?

Because it's new to me either. What can the result of writing an analytic number theory algorithm by Rust, both of which I'm not familiar with, be?

Unfortunately, after writing these code, I still don't think I've learnt Rust. Some of the features confuse me. For example, can anyone pointing me out the correct way to enable `a + b` where `a` is a `T` and `b` is `Complex<T>` for a custom float type `T`? We can't `impl<T> Add<Complex<T>> for T`  due to the orphan rule, and `num::Complex` does this by using macros, which are not exported.

### Why [Platt] instead of [Büthe] or [FKBJ]?

Because they seems too difficult to me. Welcomed by the fancy Greek letters and subscripts, I clicked X immediately. 

[Platt] seems to be quite straightforward, especially given that I've read [Galway] and understood the high-level idea. I appreciate a lot for how easy to follow [Galway] is. Without it, I wouldn't ever have the idea to implement such an algorithm, probably. (Perhaps because it's a thesis instead of a paper?) 


# LMFDB

Using LMFDB is kind of cheating. The algorithm needs all non-trivial zeros of $\zeta(\frac{1}{2} + it)$ for $t \leq \tilde O(x^{1/2})$ to compute $\pi(x)$ in $\tilde O(x^{1/2})$ time. (To make life easier, I assume RH holds. ) However, a table of $\{\pi(k^2)\}_{k^2 \leq x}$ also enables computing $\pi(x)$ in $\tilde O(x^{1/2})$ time, with a much more trivial algorithm. The only point is that the table of $\{\pi(k^2)\}_{k^2 \leq x}$ might be very hard to get :( I don't know any algorithm which can output such a table in $N^{o(1)}$ time, while it seems that the non-trivial zeros of $\zeta(\frac{1}{2} + it)$ for $t \leq T$ can be found in $\tilde O(T)$ time.

Finding all non-trivial zeros of $\zeta$ seems to be another line of work. I tried to read [Platt2] but unfortunately didn't understand it. [Gourdon] also seems promising and I have some preliminary code for that. 

# Float data type

I've implemented my own double-double data type. Although there are many existing libraries, like [TwoFloat](https://github.com/ajtribick/twofloat), I'd like to invent a new wheel for my own needs. With that being said, I appreciate their work and my implementations are heavily inspired by them. Kudos to them. 

As mentioned [here](http://www.math.uni-bonn.de/people/jbuethe/topics/AnalyticPiX.html), 

>Our implementation focuses rather on speed, using a 128 Bit fixed-point data type developed by J. Franke.

so I think double-double suffices for this project. I'm surprised to see that Jens Franke, who's Jan Büthe's advisor, developed the data type. Perhaps it's from his other projects. 

# Heuristic to decide the interval to calculate $\Delta$

We might shorten the interval by applying some heuristics based on Cramér's random model.

For brevity, we define $f(u) = \phi(u) - [u \leq x]$, i.e., $f(u) = \phi(u)$ for $u > x$ and $f(u) = \phi(u) - 1$ for $u \leq x$.

Given a real $d > 0$,  define $U_L = [1, xe^{-d\lambda}], U_R = [xe^{d \lambda}, \infty)$ and $U = U_L \cup U_R$. We'd like to sieve all primes in $(xe^{-d\lambda}, xe^{d \lambda})$ to compute $\Delta$ and estimate the truncation error $S = \sum_{p \in U}f(p)$. Note that $\lambda = \tilde O(x^{-1/2})$ and $d = \tilde O(1)$, so $x e^{d\lambda} = x - \tilde O(x^{1/2})$ and $x e^{d\lambda} = x + \tilde O(x^{1/2})$.

By Cramér's random model, we assume
$$
S = \sum_{p \in U} f(p) \approx  \sum_{u \in U} \frac{f(u)}{\ln u},
$$
One way to improve the approximation is to add an error bar, which is done by studying the behavior of a stochastic estimation $\hat S$:
$$
\hat S = \sum_{u \in U} X_u f(u), \quad X_u \sim \text{Bernoulli}(1/\ln u).
$$
We use Bernstein's inequality to approximate $S = \mathbb{E} [\hat S] + t$ by solving $t$ for $\epsilon = \exp(-20)$ in
$$
\exp\left( -\frac{\frac{1}{2} t^2}{\text{Var}[\hat S] + \frac{1}{3} M t} \right) \leq \epsilon, \quad M := \max_{u \in U} |f(u)| = \Phi(d).
$$
Let $q = 1/\ln x$, so $q \approx 1/\ln u$ for reasonable $u \in U$, i.e., those with not-too-small $f(u)$.

Next, we approximate $\mathbb{E}[\hat S]$:
$$
\mathbb{E}[\hat S] = \sum_{u \in U} \frac{f(u)}{\ln u} \approx \sum_{u \in U} q f(u) \approx q (I_r - I_l),
$$
where
$$
I_r = x \lambda \int_{d}^{\infty} \Phi(t) e^{t\lambda} \d t, \quad I_l = x \lambda \int_{d}^\infty \Phi(t) e^{-t\lambda} \d t.
$$
The last approximation is similar to [Section 3.2, Galway] and both $I_l$ and $I_r$ have a closed form solution. Unlike [Galway], we are making use of the fact that $f(u) < 0$ for $u < x$ and $f(u) > 0$ for $u > x$ and the sum of them might cancel.

The variance of $\hat S$ can be approximated by:
$$
\text{Var}[\hat S] = \sum_{u \in U} f(u)^2 \frac{1}{\ln u} (1 - 1/\ln u) \approx q(1-q) \sum_{u \in U} f(u)^2 \leq q(1-q) M \sum_{u \in U} |f(u)| \approx M q(1-q) (I_l + I_r).
$$
Finally, we minimize $d$ so that $S \leq 0.24$. In this way, we can get a shorter interval to sieve at a loss of guarantee.

In practice, when $x = 10^{11}$ and $\lambda = 3 \times 10^{-5}$, [Galway] predicts an interval of length 3e7 while this predicts an interval of length 2e7, while the difference between two sums is around 0.0112.

[Büthe] provides a rigorous way to reduce the interval, but I don't understand... [Theorem 4.3, Büthe2] seems pretty interesting and might be related to the heuristics I used. 

# Integrating $\hat\phi(s)$

Here we'd like to integrate $\hat\phi(s) = \frac{x^s}{s} \exp(\lambda^2 s^2 / 2)$. We basically follow the method in [Section 6, Platt] but with a small modification. We refer the readers to [Platt] for more details.

The high level idea is that: We express $\hat\phi(s_0 + ih) = \hat\phi(s_0) f(z) \exp(z)$ for some complex number $z = wh$ and function $f(z)$, and find a polynomial $P(z)$ to approximate $f(z)$ locally, then we apply integration by parts repeatedly:
$$
\int P(z) \exp(z) \d z = \exp(z) \sum_{k=0}^\infty P^{(k)}(z) (-1)^k = \exp(z) Q(z), \quad Q(z):=\sum_{k=0}^\infty P^{(k)}(z)(-1)^k.
$$
[Platt] chooses the following form
$$
\begin{aligned}
\hat\phi(s_0 + ih) & = \frac{x^{s_0 + ih}}{s_0 + ih} \exp(\lambda^2 (s_0 + ih)^2 / 2) \\
 & = \hat\phi(s_0) \frac{x^{ih}}{1 + \frac{ih}{s0}} \exp(\lambda^2 (-h^2+2ihs_0)/2) \\
 & = \hat\phi(s_0) \frac{\exp(-\lambda^2 h^2/2)}{1 + \frac{i}{s_0}h} \exp(ih (\lambda^2 s_0 + \ln x)),
\end{aligned}
$$
while we choose
$$
\hat\phi(s_0 + ih)  = \hat\phi(s_0) \frac{\exp(-\lambda^2 h^2/2 + ih \lambda^2 s_0)}{1 + \frac{i}{s_0}h} \exp(ih \ln x).
$$
The benefit of ours is that $ih\ln x$ has only imaginary part, so it's faster to compute the exponential function. This can speed it up a little bit (5% perhaps) when $x$ is not too large that LMFDB has enough zeros of $\zeta(s)$.

We pick up $\hat t_{1, \dots, m}$ such that for any $t$ of interest, we can find a $\hat t_j$ *around* it. More specifically, each $\hat t_j$ can work for $t \in [\hat t_j - r_j, \hat t_j + r_j]$, so we just want to make sure that $\bigcup [\hat t_j - r_j, \hat t_j + r_j]$ covers $[0, U]$ where $U$ is the upper limit of integral. To achieve the same approximation error, the radius $r_i$ is proportional to $|\hat t_j|$, so we only need $m = O(\log U)$.

## Hybrid Precision Integration

In this subsection, we'd like to use low precision data types to speed up the term $\sum_\rho \hat\Phi(\rho)$.

Assume each $\hat\Phi(\sigma + it)$ is calculated with an relative error at most $\delta$, the absolute error of the summation is then bounded by
$$
\delta \sum_{0 < \Im \rho < T}  |\hat\Phi(\rho)|,
$$
which turns out to to be $O(\delta x^{1/2} \ln x)$: By [Lemma A.2, Platt], for $\rho = \sigma + it$ with $t > 1$,
$$
|\hat\Phi(\rho)| \leq B(\sigma, t) \leq \frac{x^\sigma}{t \ln x} + \frac{1}{\lambda^2 t^2 x},
$$
so we approximate $\sum_{1 < \Im \rho < T} |\hat\Phi(\rho)|$ by an integral, considering the density of $\zeta$ zeros at height $t$ is $O(\ln t)$ and $T = O(x)$,
$$
\sum_{1 < \Im \rho < T} |\hat\Phi(\rho)| \leq \int_{1}^T \left( \frac{x^\sigma}{t \ln x} + \frac{1}{\lambda^2 t^2 x} \right) \d N(t) = \frac{x^\sigma}{\ln x} \int_1^T \frac{1}{t} \d N(t) + \frac{1}{\lambda^2 x} \int_1^T \frac{1}{t^2} \d N(t) = O(x^{1/2} \ln x).
$$
More rigorous result can be obtained similar to [Lemma A.3, Platt]. 

In practice, the term
$$
\sum_{\Im \rho > 0} |\hat \Phi(\rho)| \approx 5.5 \times 10^8,
$$
when $x = 10^{18}, \lambda = 5 \times 10^{-8}$, so it's totally fine for `f64`. 

So we pick $\hat t_{1, \dots, m}$ and compute $\hat\Phi(\sigma + i \hat t_{j})$ for every  $j$ using high precision. Then we approximate $\hat\phi(s_0 + ih)$ locally as mentioned before, for each $s_0 = \sigma + i \hat t_j$ using `f64`. When we want to query $\hat\Phi(\sigma + it)$ for any $t$, we find the last $\hat t_j \leq t$ and use the relationship:
$$
\hat \Phi(\sigma + it) = \hat\Phi(\sigma + i\hat t_j) + \int_t^{\hat t_j} \hat\phi(\sigma + i h) \d h,
$$
the latter of which is computed as before, but with `f64`. In practice, this method often leads to a 8x speedup when computing the integral. 

## Reducing relative error

The bottleneck of reducing the relative error $\delta$ is trigonometric functions: $\sin(x)$ has a relative error of $O(|x|)$. So we simply first reduce any $z$ to $z \bmod 2\pi$ in high precision, and then compute $\exp(iz)$ in `f64`. Numerical experiment shows that the simple trick can reduce $\delta$ to $10^{-14}$ (in a different scenario). 

The small relative error might sound unrealistic at first, as the derivative of $\hat\Phi(s)$ is $\hat\phi(s)$, which has norm of $O(x^{1/2})$, so when we truncate a $z$ into `f64`, the error should be $2^{-53}x^{1/2}$, which is way larger than $10^{-14}$. The reason is that the approximation of the integral has the form of $Q(z) \exp(z)$, and truncating $z$ to `f64` leads to large relative error in $\exp(z)$, especially when $z$ is pure imaginary, i.e., $\exp(z + \delta z)$ might different a lot from $(1 + \delta) \exp(z)$. However, $Q(z)$ is much numerical stable in most cases:
$$
Q(z + \delta z) = \sum_{k=0}^\infty P^{(k)}(z + \delta z)(-1)^k \approx \sum_{k=0}^\infty (-1)^k \left( P^{(k)} (z) + \delta P^{(k+1)}(z) \right) = Q(z) + \delta (P(z) - Q(z)).
$$
I guess $P(z)$ dominates other terms in $Q(z)$ as the function it approximates has fast converging power series, so $P(z) \approx Q(z)$ and $Q(z + \delta z) \approx Q(z)$, which means unlike $\exp$, $Q$ doesn't magnify the relative error. Note that this doesn't conflict with the fact that we need high precision of $\zeta$ zeros, as $\exp$ requires so. 

# Computing $\Delta$ 

Recall that
$$
\Delta = \sum_{L \leq p \leq R} f(p) = \sum_{L \leq p \leq R} (\phi(p) - [p \leq x]).
$$
For a prime $p$, if we use `f64` to store it, there might be a precision loss. Moreover, inaccuracy also be introduced when calculating $f(p)$. Let $\delta$ be the maximum absolute error of the output of $f(p)$. Then the total absolute error is at most $(R - L + 1) \delta$. 

To avoid precision loss of converting `u64` to `f64`, we calculate $\tilde f(t) = f(x + t)$, with $t = O(\lambda x)$. 

To calculate $\tilde f(t)$ fast: we simply use the order-2 Taylor polynomial at $t_0$. 
$$
g(t) = \tilde f(t_0) + (t - t_0) \tilde f'(t_0) + \frac{1}{2}(t - t_0)^2 \tilde f''(t_0).
$$
where
$$
\begin{aligned}
\tilde f'(t) & = -\frac{C}{x + t}\lambda^{-1}, \\
\tilde f''(t) & = \frac{C}{(x+t)^2} (\lambda^{-1} + \rho \lambda^{-2}), \\
\end{aligned}
$$
for $\rho = \lambda^{-1} \ln (1+t/x), C = -\frac{\exp(-\rho^2 /2)}{\sqrt{2 \pi}}$. Note that we sometimes find a new $t_0$ and build a new order-2 Taylor polynomial. So we need to determine when the approximation is good. More specifically, we'd like to know what's the maximum $w > 0$, such that, it hold that have $\left|g(t) - \tilde f(t)\right| \leq \epsilon$ for all $t$ in $[t_0 - w, t_0 + w]$. By using the Lagrange remainder, we have
$$
|g(t) - \tilde f(t)| \leq \frac{1}{6} w^3 \max_t |\tilde f'''(t)|,
$$
where
$$
\tilde f'''(t) = -\frac{C}{(x+t)^3} (2 \lambda^{-1} + 3 \rho \lambda^{-2} - (1 - \rho^2) \lambda^{-3}).
$$
Note that $\exp(-\rho^2 / 2) \rho \geq -0.63$ and $\exp(-\rho^2 / 2) \rho^2 \geq 0$ for every $\rho$, we can then bound $|\tilde f'''(t)|$ (for most $\lambda$):
$$
|\tilde f'''(t)| \leq \frac{-2 \lambda^{-1} + 1.8 \lambda^{-2} + \lambda^{-3}}{x^3 \sqrt{2 \pi}}.
$$

We limit $w$ to be smaller than $2^{21}$, so that for all primes $p$ in $[t_0, t_0 + w]$, the sum of $(p - t_0)^2$ fits into a 64-bit integer:
$$
\sum_{t_0 \leq p < t_0 + w} (p - t_0)^2 < \frac{1}{3} w^3 < 2^{63},
$$
so that we don't need 128-bit integers. Making most operations in integer type slightly increase the speed of calculating $\phi$ (10%), although the bottleneck is sieving primes. It turns out that it's fine to use high-precision data type to compute $\Delta$, as most operations are integer operations. High-precision data type is only used to compute the coefficients $\hat f(t_0), \hat f'(t_0), \hat f''(t_0)$, which are re-computed once per nearly 25k primes (when $x = 10^{18}$ and $\lambda \approx 4.55 \times 10^{-8}$). 

# Sieving primes

To make life even easier, I use the highly-optimized library [primesieve](https://github.com/kimwalisch/primesieve). There are other alternatives, e.g., the `primal` crate. But it seems that `primal` is not segmented, and is slightly slower. 

`primesieve` prevents us from computing $\pi(x)$ for $x \geq 2^{64}$, although there are other factors, e.g., precision of double-double, saving zeros of $\zeta(s)$, 128-bit integer date type, and the most importantly, computational resources. The algorithm can run in parallel easily: The most time-consuming part is sieving primes, and segmented sieve runs in parallel perfectly. 

# References

+ [[Platt](https://arxiv.org/abs/1203.5712)] : David Platt, “Computing $\pi(x)$ Analytically.”
+ [[Platt2](https://www.ams.org/journals/mcom/2017-86-307/S0025-5718-2017-03198-7/S0025-5718-2017-03198-7.pdf)] : David Platt, “Isolating some non-trivial zeros of zeta”
+ [[FKBJ](https://www.ams.org/journals/mcom/2017-86-308/S0025-5718-2017-03038-6/S0025-5718-2017-03038-6.pdf)] : Franke et al., “A Practical Analytic Method for Calculating $\pi (x)$.”
+ [[Büthe](https://arxiv.org/abs/1410.7008)] : Jan Büthe, “An Improved Analytic Method for Calculating $\pi(x)$.”
+ [[Büthe2](https://arxiv.org/abs/1410.7561)] : Jan Büthe, "A Brun-Titchmarsh inequality for weighted sums over prime numbers"
+ [[Galway](https://faculty.math.illinois.edu/~galway/PhD_Thesis/thesis-twoside.pdf)] : William Galway, "Analytic Computation of the Prime-Counting Function".
+ [[Gourdon](http://numbers.computation.free.fr/Constants/Miscellaneous/zetazeros1e13-1e24.pdf)] : Xavier Gourdon, "The $10^{13}$ first zeros of the Riemann Zeta function, and zeros computation at very large height"

I also briefly summarized these papers [here](https://www.roosephu.xyz/2021/03/08/analytic-pi/) and [here](https://www.roosephu.xyz/2021/03/08/analytic-pi-2/) (in Chinese). 


